# Web Scraping

by: Angela Lisanthoni (21083010032)

1. Mengambil link untuk setiap kategori
2. lakukan looping untuk tiap link kategori
3. looping dilakukan untuk mendapatkan link berita di kategori tersebut
4. di setiap link berita, diambil judul, tanggal+waktu, penulis, isi

## Cara 1: Dengan filter link yang memiliki format berbeda

In [4]:
from bs4 import BeautifulSoup
import requests
import argparse
import random
import pandas as pd
import time
import numpy as np

In [5]:
def DataframeBerita(url):
    kategori=[]
    linknya=[]
    kategori2 = []
    linknya2 = []
    judul = []
    penulis = []
    tanggal_waktu = []
    isi = []
    
    """Digunakan untuk mengambil link per kategorinya terlebih dahulu"""
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")

    hasil_kategori = []
    nav_item = soup.find_all(class_="nav__item") #nama_class

    for item in nav_item:
        link_elem = item.find('a')
        if link_elem:
            link = link_elem.get('href')
            hasil_kategori.append(link)

    #filter 1: mengambil yang dimulai dengan'https://news.detik.com/
    filtered_links = [link for link in hasil_kategori if link.startswith('https://news.detik.com/')]

    #filter 2: hapus nilai yang panjangnya lebih dari yang ditentukan
    jumlah = len('https://news.detik.com/') + 15
    filtered_links = [link for link in filtered_links if len(link) < jumlah]

    #filter 3: hapus duplikasi nilai
    filtered_links = list(set(filtered_links))
    
    """Dari link tiap kategori, diambil link beritanya"""
    for i in filtered_links:
        url1 = i
        response = requests.get(url1)
        soup = BeautifulSoup(response.text, "html.parser")
        link_artikel = soup.find_all(class_="ph_newsfeed_d article_inview list-content__item") #nama_class

        for link in link_artikel:
            kategori.append(i)
            linknya.append(link.get("i-link"))

    kategori = [elemen.replace('https://news.detik.com/', '') for elemen in kategori]
    
    array_gabungan = list(zip(kategori, linknya))

    #tidak bisa di scrape, bukti ada di paling bawah sehingga perlu difilter
    filtered = [(kategori1, link1) for kategori1, link1 in array_gabungan if not link1.startswith('https://20.detik.com/')]
    filtered = [(kategori1, link1) for kategori1, link1 in filtered if not link1.startswith('https://news.detik.com/x/')]
    filtered = [(kategori1, link1) for kategori1, link1 in filtered if not link1.startswith('https://news.detik.com/foto-news/')]
    for item in filtered:
        kategori2.append(item[0])
        linknya2.append(item[1])
    
    """Dari tiap berita, diambil judul, penulis, tanggal dan waktu, serta isi"""
    for j in linknya2:
        url = j
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        #judul
        judul_artikel = soup.find(class_="detail__title").get_text(strip=True) #nama_class
        judul.append(judul_artikel)

        #penulis
        penulis_artikel = soup.find(class_="detail__author").get_text(strip=True) #nama_class
        penulis.append(penulis_artikel)
        penulis = [elemen.replace(' -detikNews', '') for elemen in penulis]

        #tanggal waktu
        tanggal_waktu_artikel = soup.find(class_="detail__date").get_text(strip=True) #nama_class
        tanggal_waktu.append(tanggal_waktu_artikel)

        #isi
        isi_artikel = soup.find(class_="detail__body-text itp_bodycontent").get_text(strip=True) #nama_class
        isi.append(isi_artikel)
        
    """Membuat Dataframe dari array - array yang terbentuk"""
    df = pd.DataFrame({
        'no Urut': range(1, len(kategori2) + 1),
        'Kategori': kategori2,
        'Link': linknya2,
        'Judul': judul,
        'Penulis': penulis,
        'Tanggal dan Waktu': tanggal_waktu,
        'isi': isi
    })

    df['Kategori'] = df['Kategori'].replace('', 'home')

    return df


In [6]:
DataframeBerita("https://news.detik.com/")

,no Urut,Kategori,Link,Judul,Penulis,Tanggal dan Waktu,isi
0,1,kolom,https://news.detik.com/kolom/d-6736421/cawe-ca...,"""Cawe-Cawe"" Urusan Pilpres",Ahmad Farisi,"Kamis, 25 Mei 2023 13:30 WIB",Jakarta-Presiden adalah jabatan publik. Secara...
1,2,kolom,https://news.detik.com/kolom/d-6736336/golkar-...,Golkar dan Agenda Politik 2024,Ahmad Doli Kurnia Tandjung,"Kamis, 25 Mei 2023 11:00 WIB",Jakarta-Kurang dari sepuluh bulan lagi kita ak...
2,3,kolom,https://news.detik.com/kolom/d-6736270/pemilu-...,Pemilu di Tengah Krisis Iklim,Zuhad Aji Firmantoro,"Kamis, 25 Mei 2023 08:50 WIB",Jakarta-Tahun 2024 pesta demokrasi yang sangat...
3,4,kolom,https://news.detik.com/kolom/d-6736243/cuaca-e...,Cuaca Ekstrem Menuju Pesta Demokrasi 2024,Muhammad Naufal,"Rabu, 24 Mei 2023 15:30 WIB",Jakarta-Masyarakat Indonesia saat ini tengah d...
4,5,kolom,https://news.detik.com/kolom/d-6736119/capres-...,Capres: Petugas Partai atau Petugas Rakyat?,Agus Riewanto,"Rabu, 24 Mei 2023 14:10 WIB",Jakarta-Akhir-akhir ini wacana tentang calon p...
...,...,...,...,...,...,...,...
289,290,jabodetabek,https://news.detik.com/berita/d-6738077/ternya...,Ternyata Kapolda Metro Ditelepon Mahfud Sebelu...,Devi Puspitasari,"Kamis, 25 Mei 2023 13:10 WIB",Jakarta-Kasusistri korban KDRTmalah jadi tersa...
290,291,jabodetabek,https://news.detik.com/berita/d-6738028/teperg...,"Tepergok Mencuri, Maling di Jakut Dikejar Mass...",Wildan Noviansah,"Kamis, 25 Mei 2023 12:46 WIB","Jakarta-Seorangmalingdi wilayah Tanjung Priok,..."
291,292,jabodetabek,https://news.detik.com/berita/d-6737991/35-wna...,35 WNA Ilegal di Jakut Dijaring Buntut 2 Nenek...,Jabbar Ramdhani,"Kamis, 25 Mei 2023 12:30 WIB",Jakarta-Razia warga negara asing (WNA) ilegal ...
292,293,jabodetabek,https://news.detik.com/berita/d-6737989/pihak-...,Pihak Istri Korban KDRT Sebut Suami Pemakai Na...,Devi Puspitasari,"Kamis, 25 Mei 2023 12:29 WIB","Depok-Pria inisial B, suami yang dilaporkan me..."


## Cara 2: Mendefinisikan tiap link yang didapatkan tanpa ada pembuangan link

In [103]:
from bs4 import BeautifulSoup
import requests

"""LINK PER KATEGORI"""
link1 = [] 
kategori = []
url = 'https://news.detik.com/' 
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

links = soup.find_all('a')
for link in links:
    if link.get('dtr-evt') == 'header' and link.get('dtr-act') == 'first navbar':
        if link.get('dtr-ttl') == "logo detikcom":
            continue
        #print(link.get('dtr-evt'))
        #print(link.get('href'))
        link1.append(link.get('href'))
        kategori.append(link.text.strip())
        
array_gabungan = list(zip(kategori, link1))
#print(link1)
#print(kategori)
print(array_gabungan)

[('Home', 'https://news.detik.com/'), ('Berita', 'https://news.detik.com/berita'), ('Jabodetabek', 'https://news.detik.com/jabodetabek'), ('Internasional', 'https://news.detik.com/internasional'), ('Hukum', 'https://news.detik.com/hukum'), ('detikX', 'https://news.detik.com/x/'), ('Kolom', 'https://news.detik.com/kolom'), ('Blak Blakan', 'https://news.detik.com/blak-blakan'), ('Pro Kontra', 'https://news.detik.com/pro-kontra'), ('Infografis', 'https://news.detik.com/infografis'), ('Foto', 'https://news.detik.com/foto'), ('Video', 'https://news.detik.com/video'), ('Indeks', 'https://news.detik.com/indeks')]


In [104]:
result = {}

for category, url in array_gabungan:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    link_artikel = soup.find_all(class_="ph_newsfeed_d article_inview list-content__item")

    for j in link_artikel:
        link = j.get("i-link")
        if link not in result:
            result[link] = category

result = list(result.items())
result = [(link, category) for category, link in result]


In [105]:
len(result)

213

In [128]:
"""JUDUL, PENULIS, TANGGAL, ISI"""
judul = []
penulis = []
tanggal_dan_waktu = []
isi = []
category1 = []
url_berita1 = []

for category, url_berita in result:
    category1.append(category)
    url_berita1.append(url_berita)
    url = url_berita
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    print(url)
    
    #judul
    judul_artikel = soup.find(class_="detail__title")
    if judul_artikel is None:
        judul_artikel= soup.find('meta', property='og:title')
        #print(judul_artikel.get('content'))
        judul.append(judul_artikel.get('content'))
    #print(judul_artikel.get_text(strip=True))
    judul.append(judul_artikel.get_text(strip=True))
    judul = list(filter(None, judul))

    #penulis
    penulis_artikel = soup.find(class_="detail__author")
    if penulis_artikel is None:
        penulis_artikel = soup.find('meta', attrs={'name': 'author'})
        #print(penulis_artikel.get('content'))
        penulis.append(penulis_artikel.get('content'))
    if penulis_artikel is not None:
        penulis_artikel = penulis_artikel.text.split('-')[0].strip()
        #print(penulis_artikel)
        penulis.append(penulis_artikel)
    penulis = list(filter(None, penulis))

    #tanggal waktu
    tanggal_waktu_artikel = soup.find(class_="detail__date") #nama_class    
    if tanggal_waktu_artikel is None:
        tanggal_waktu_artikel = soup.find(class_='date')
        if tanggal_waktu_artikel is None:
            tanggal_waktu_artikel = soup.find('div', class_='caption')
            tanggal_waktu_artikel = tanggal_waktu_artikel.find('span')
            #print(tanggal_waktu_artikel.get_text(strip=True))
            tanggal_dan_waktu.append(tanggal_waktu_artikel.get_text(strip=True))
        else:
            #print(tanggal_waktu_artikel.get_text(strip=True))
            tanggal_dan_waktu.append(tanggal_waktu_artikel.get_text(strip=True))
        
    elif tanggal_waktu_artikel is not None:
        tanggal_waktu_artikel = tanggal_waktu_artikel
        #print(tanggal_waktu_artikel.get_text(strip=True))
        tanggal_dan_waktu.append(tanggal_waktu_artikel.get_text(strip=True))

    
    #isi
    isi_artikel = soup.find(class_="itp_bodycontent") 
    if isi_artikel is None:
        isi_artikel = soup.find(class_="detail__body-text") 
        if isi_artikel is None:
            isi_artikel = soup.find(class_="column full body_text") 
            #print(isi_artikel.get_text(strip=True))
            isi.append(isi_artikel.get_text(strip=True))
        else:
            #print(isi_artikel.get_text(strip=True))
            isi.append(isi_artikel.get_text(strip=True))
            
    elif isi_artikel is not None:
        #print(isi_artikel.get_text(strip=True))
        isi.append(isi_artikel.get_text(strip=True))



https://news.detik.com/pemilu/d-6772468/mk-putuskan-sistem-pemilu-besok-kpu-pastikan-tahapan-pemilu-tak-terganggu
https://20.detik.com/detikupdate/20230614-230614105/ratusan-korban-terdampak-kebakaran-di-denpasar-direlokasi-ke-rumah-kos
https://news.detik.com/pemilu/d-6772461/menanti-putusan-sistem-pemilu-kammi-harap-mk-tak-bunuh-demokrasi
https://news.detik.com/berita/d-6772456/anggota-tni-al-di-kepri-diperiksa-usai-rumahnya-jadi-penampungan-pmi-ilegal
https://news.detik.com/berita/d-6772453/jadi-sengketa-mobil-alphard-milik-jessica-iskandar-diblokir
https://news.detik.com/berita/d-6772446/jokowi-bakal-cawe-cawe-demi-putri-ariani-di-americas-got-talent
https://20.detik.com/detikupdate/20230614-230614102/nasdem-tegaskan-terima-siapapun-cawapres-yang-dipilih-anies
https://news.detik.com/berita/d-6772444/mendes-pdtt-ungkap-upaya-berdayakan-perempuan-sesuai-sdgs-desa
https://news.detik.com/berita/d-6772422/guru-sd-diduga-lecehkan-murid-disdik-bogor-serahkan-proses-hukum-ke-polisi
https://

In [129]:
"""MEMBUAT DATAFRAME"""
import pandas as pd
df = pd.DataFrame({
        'Kategori': category1,
        'Link': url_berita1,
        'Judul': judul,
        'Penulis': penulis,
        'Tanggal dan Waktu': tanggal_dan_waktu,
        'isi': isi
    })

df      

,Kategori,Link,Judul,Penulis,Tanggal dan Waktu,isi
0,Home,https://news.detik.com/pemilu/d-6772468/mk-put...,"MK Putuskan Sistem Pemilu Besok, KPU Pastikan ...",Anggi Muliawati,"Rabu, 14 Jun 2023 16:15 WIB",Jakarta-Mahkamah Konstitusi (MK) akan menggela...
1,Home,https://20.detik.com/detikupdate/20230614-2306...,Ratusan Korban Terdampak Kebakaran di Denpasar...,I Nyoman Adhisthaya Sawitra,"603 Views | Rabu, 14 Jun 2023 16:00 WIB",Pemerintah Kota (Pemkot) Denpasar akan merelok...
2,Home,https://news.detik.com/pemilu/d-6772461/menant...,"Menanti Putusan Sistem Pemilu, KAMMI Harap MK ...",Danu Damarjati,"Rabu, 14 Jun 2023 16:12 WIB","Jakarta-Besok, wakil Tuhan yangberkantor di Jl..."
3,Home,https://news.detik.com/berita/d-6772456/anggot...,Anggota TNI AL di Kepri Diperiksa Usai Rumahny...,Alamudin Hamapu,"Rabu, 14 Jun 2023 16:10 WIB",Jakarta-Polisi Militer AL (Pomal) Lantamal IV ...
4,Home,https://news.detik.com/berita/d-6772453/jadi-s...,"Jadi Sengketa, Mobil Alphard Milik Jessica Isk...",Wildan Noviansah,"Rabu, 14 Jun 2023 16:10 WIB",Jakarta-Polisi masih memburuChristopher Sfefan...
...,...,...,...,...,...,...
208,Pro Kontra,https://news.detik.com/pro-kontra/d-6162839/ni...,"Nikah Beda Agama Diuji ke MK, Anda Setuju Atau...",Rakhmad Hidayatulloh Permana,"Selasa, 05 Jul 2022 12:14 WIB",Jakarta-Pemerintah menolak melegalkanpernikaha...
209,Pro Kontra,https://news.detik.com/pro-kontra/d-6142950/ca...,Cak Imin Vs Yenny Wahid (Lagi),Rakhmad Hidayatulloh Permana,"Kamis, 23 Jun 2022 14:34 WIB","Jakarta-Putri Gus Dur, Zaanuba Arifah Chafsoh ..."
210,Pro Kontra,https://news.detik.com/pro-kontra/d-6138050/rk...,"RKUHP Ancam Bui 'Demo Tanpa Pemberitahuan', Se...",Tim detikcom,"Selasa, 21 Jun 2022 06:55 WIB",Jakarta-Draf Rancangan Kitab Undang-Undang Huk...
211,Pro Kontra,https://news.detik.com/pro-kontra/d-6136938/ny...,"'Nyinyirin' Presiden di Medsos Dibui 4,5 Tahun...",Danu Damarjati,"Senin, 20 Jun 2022 15:19 WIB",Jakarta-Di dalam draf Rancangan Kitab Undang-U...
